In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
with open('cube_history.txt', 'r') as f:
    history = ''
    lines = f.readlines()
    for line in tqdm(lines):
        if ': true' in line:
            line = line.replace(': true', ': True')
        
        if ': false' in line:
            line = line.replace(': false', ': False')
        
        history += line
    
    del lines, line
    f.close()

100%|██████████| 23361/23361 [00:03<00:00, 6115.85it/s] 


In [3]:
history = eval(history)['cube_histories']

+ id  
+ character_name  
+ create_date  
+ cube_type  
+ item_upgrade_result  
+ miracle_time_flag  
+ item_equip_part  
+ item_level  
+ target_item  
+ potential_option_grade  
+ additional_potential_option_grade  
+ upgradeguarantee  
+ upgradeguaranteecount  
+ before_potential_options  
+ before_additional_potential_options  
+ after_potential_options  
+ after_additional_potential_options

# 등급업만 보기

In [4]:
for result_dict in history:
    result_dict['before_potential_options'] = result_dict['before_potential_options'][0]['grade']
    del result_dict['after_potential_options']
    del result_dict['before_additional_potential_options']
    del result_dict['after_additional_potential_options']
    del result_dict['id'], result_dict['character_name']

In [5]:
history = pd.DataFrame(history)
history.create_date = pd.to_datetime(history.create_date).dt.strftime('%Y-%m-%d %H:%M:%S')
history = history.sort_values(by='create_date').reset_index(drop=True)

In [6]:
history

,create_date,cube_type,item_upgrade_result,miracle_time_flag,item_equip_part,item_level,target_item,potential_option_grade,additional_potential_option_grade,upgradeguarantee,upgradeguaranteecount,before_potential_options
0,2023-08-24 00:00:00,장인의 큐브,실패,이벤트 적용되지 않음,귀고리,130,지옥의 불꽃,유니크,노멀,False,0,유니크
1,2023-08-24 00:00:02,장인의 큐브,실패,이벤트 적용되지 않음,귀고리,130,지옥의 불꽃,유니크,노멀,False,0,유니크
2,2023-08-24 00:00:04,장인의 큐브,실패,이벤트 적용되지 않음,귀고리,130,지옥의 불꽃,유니크,노멀,False,0,유니크
3,2023-08-24 00:00:06,장인의 큐브,실패,이벤트 적용되지 않음,귀고리,130,지옥의 불꽃,유니크,노멀,False,0,유니크
4,2023-08-24 00:00:08,장인의 큐브,실패,이벤트 적용되지 않음,귀고리,130,지옥의 불꽃,유니크,노멀,False,0,유니크
...,...,...,...,...,...,...,...,...,...,...,...,...
514,2023-08-24 13:19:46,블랙 큐브,실패,이벤트 적용되지 않음,무기,160,앱솔랩스 엑스,유니크,노멀,False,23,유니크
515,2023-08-24 13:19:47,블랙 큐브,실패,이벤트 적용되지 않음,무기,160,앱솔랩스 엑스,유니크,노멀,False,24,유니크
516,2023-08-24 13:19:49,블랙 큐브,실패,이벤트 적용되지 않음,무기,160,앱솔랩스 엑스,유니크,노멀,False,25,유니크
517,2023-08-24 13:19:51,블랙 큐브,실패,이벤트 적용되지 않음,무기,160,앱솔랩스 엑스,유니크,노멀,False,26,유니크


In [7]:
history.drop(columns=['item_equip_part', 'item_level', 'target_item', 'additional_potential_option_grade'], inplace=True)

history.miracle_time_flag = history.miracle_time_flag.map({
    '이벤트 적용되지 않음': False,
    '이벤트 적용중': True
})
history.item_upgrade_result = history.item_upgrade_result.map({
    '실패': False,
    '성공': True
})

grade_mapper = {
    '레어': 'rare',
    '에픽': 'epic',
    '유니크': 'unique',
    '레전드리': 'legendary'
}
history.potential_option_grade = history.potential_option_grade.map(grade_mapper)
history.before_potential_options = history.before_potential_options.map(grade_mapper)

history.columns = ['date', 'cube', 'upgrade_result', 'miracle', 'after_grade', 'ceiling', 'ceiling_count', 'before_grade']
history = history.loc[:, ['date', 'cube', 'before_grade', 'after_grade', 'upgrade_result', 'ceiling', 'ceiling_count', 'miracle']]

In [8]:
history.head()

,date,cube,before_grade,after_grade,upgrade_result,ceiling,ceiling_count,miracle
0,2023-08-24 00:00:00,장인의 큐브,unique,unique,False,False,0,False
1,2023-08-24 00:00:02,장인의 큐브,unique,unique,False,False,0,False
2,2023-08-24 00:00:04,장인의 큐브,unique,unique,False,False,0,False
3,2023-08-24 00:00:06,장인의 큐브,unique,unique,False,False,0,False
4,2023-08-24 00:00:08,장인의 큐브,unique,unique,False,False,0,False


In [10]:
history.loc[(history.ceiling_count == 0) & (history.cube == '블랙 큐브'), :]

,date,cube,before_grade,after_grade,upgrade_result,ceiling,ceiling_count,miracle
45,2023-08-24 08:29:50,블랙 큐브,epic,unique,True,True,0,False
182,2023-08-24 11:42:13,블랙 큐브,epic,unique,True,False,0,False
491,2023-08-24 13:18:55,블랙 큐브,epic,unique,True,False,0,False
